In [1]:
from utility.MoleculeParser import MoleculeData
from utility.QMUQUBO import QMUQUBO
from utility.AnnealerOptimizer import Annealer

import logging

logging.getLogger().setLevel(logging.INFO)


In [2]:
!python --version

Python 3.7.12


In [3]:
!pip show amazon-braket-schemas
!pip show amazon-braket-sdk
!pip show amazon-braket-ocean-plugin

Name: amazon-braket-schemas
Version: 1.4.1
Summary: An open source library that contains the schemas for Amazon Braket
Home-page: https://github.com/aws/amazon-braket-schemas-python
Author: Amazon Web Services
Author-email: 
License: Apache License 2.0
Location: /Users/aoyuzhan/miniconda3/envs/d2l/lib/python3.7/site-packages
Requires: pydantic
Required-by: amazon-braket-default-simulator
Name: amazon-braket-sdk
Version: 1.9.5.post0
Summary: An open source library for interacting with quantum computing devices on Amazon Braket
Home-page: https://github.com/aws/amazon-braket-sdk-python
Author: Amazon Web Services
Author-email: 
License: Apache License 2.0
Location: /Users/aoyuzhan/miniconda3/envs/d2l/lib/python3.7/site-packages
Requires: amazon-braket-default-simulator, backoff, boltons, boto3, coverage, nest-asyncio, networkx, numpy
Required-by: amazon-braket-ocean-plugin
Name: amazon-braket-ocean-plugin
Version: 1.0.6
Summary: An open source framework for interacting with D-Wave's Ocea

In [2]:
# parameters shared by different steps
qa_task_id = "661c79bd-a01b-4c13-a5a8-8aa99ce5ec2f"
s3_bucket = f"amazon-braket-1a222675c751" # the name of the bucket
prefix = "annealer-experiment" # the name of the folder in the bucket

raw_path = './molecule-data/Aspirin.mol2'
data_path = './qmu_Aspirin_data_latest.pickle'
model_path = './qmu_Aspirin_model_latest.pickle'

# Prepare Molecule Data

In [3]:
mol_data = MoleculeData(raw_path, 'qmu')

mol_data.save("latest")

INFO:root:parse mol2 file!
INFO:root:finish save qmu_Aspirin_data_latest.pickle


'./qmu_Aspirin_data_latest.pickle'

# Build QUBO model for molecule unfolding

In [4]:
init_param = {}
method = ['pre-calc']

for mt in method:
    if mt == 'pre-calc':
        init_param[mt] = {}
        init_param[mt]['param'] = ['M', 'D', 'A', 'hubo_qubo_val']
    
qmu_qubo = QMUQUBO(mol_data, method, **init_param)

INFO:root:initial pre-calculate for constructing molecule QUBO


In [5]:
model_param = {}
# parameters
num_rotation_bond = mol_data.bond_graph.rb_num

method = 'pre-calc'
model_param[method] = {}
# model_param[method]['M'] = range(1, num_rotation_bond+1)
model_param[method]['M'] = [1]
model_param[method]['D'] = [4]
model_param[method]['A'] = [300]
model_param[method]['hubo_qubo_val'] = [200]

qmu_qubo.build_model(**model_param)


INFO:root:['4+5']
INFO:root:Construct model for M:1,D:4,A:300,hubo_qubo_val:200 6.297032038370768e-05 min


0

In [6]:
model_info = qmu_qubo.describe_model()

INFO:root:method: pre-calc
INFO:root:The model_name should be {M}_{D}_{A}_{hubo_qubo_val}
INFO:root:param: M, value {1}
INFO:root:param: D, value {4}
INFO:root:param: A, value {300}
INFO:root:param: hubo_qubo_val, value {200}


In [7]:
qmu_qubo.save("latest")

INFO:root:finish save qmu_Aspirin_model_latest.pickle


'./qmu_Aspirin_model_latest.pickle'

# Optimize to find the configuration

In [8]:
from utility.AnnealerOptimizer import Annealer
from utility.QMUQUBO import QMUQUBO
import logging

In [11]:
qmu_qubo_optimize = QMUQUBO.load(model_path)


In [12]:
model_info = qmu_qubo_optimize.describe_model()

INFO:root:method: pre-calc
INFO:root:The model_name should be {M}_{D}_{A}_{hubo_qubo_val}
INFO:root:param: M, value {1}
INFO:root:param: D, value {4}
INFO:root:param: A, value {300}
INFO:root:param: hubo_qubo_val, value {200}


In [13]:
M = 1
D = 4
A = 300
hubo_qubo_val = 200
model_name = "{}_{}_{}_{}".format(M, D, A, hubo_qubo_val)
method = "pre-calc"

qubo_model = qmu_qubo_optimize.get_model(method, model_name)

In [14]:
method = 'dwave-sa'

optimizer_param = {}
optimizer_param['shots'] = 1

sa_optimizer = Annealer(qubo_model, method, **optimizer_param)

INFO:root:use simulated annealer from dimod


In [15]:
sa_optimize_result = sa_optimizer.fit()


INFO:root:dwave-sa save to local
INFO:root:finish save sa_result.pickle


In [16]:
method = 'dwave-qa'

optimizer_param = {}
optimizer_param['shots'] = 1
optimizer_param['bucket'] = s3_bucket # the name of the bucket
optimizer_param['prefix'] = prefix # the name of the folder in the bucket
optimizer_param['device'] = "arn:aws:braket:::device/qpu/d-wave/Advantage_system4"
optimizer_param["embed_method"] = "default"

qa_optimizer = Annealer(qubo_model, method, **optimizer_param)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:root:use quantum annealer arn:aws:braket:::device/qpu/d-wave/Advantage_system4 


In [17]:
# not create annealing task, only embedding logic
qa_optimizer.embed()
# create annealing task
qa_optimize_result = qa_optimizer.fit()

INFO:root:finish save qa_result.pickle
INFO:root:dwave-qa save to s3 - 661c79bd-a01b-4c13-a5a8-8aa99ce5ec2f: None


In [18]:
qa_task_id = qa_optimizer.get_task_id()
logging.info(f"task id is {qa_task_id}")

INFO:root:task id is 661c79bd-a01b-4c13-a5a8-8aa99ce5ec2f


In [19]:
logging.info(f"dwave-sa run time {sa_optimize_result['time']}")
logging.info(f"dwave-qa run time {qa_optimize_result['time']}")

INFO:root:dwave-sa run time 0.017264842987060547
INFO:root:dwave-qa run time 6.910878896713257


# Postprocess for evaluation

#### get time for qpu

In [3]:
from utility.ResultProcess import ResultParser
import time
import logging

timestamp = time.strftime("%Y%m%d-%H")

method = "dwave-sa"
sa_param = {}
sa_param["raw_path"] = raw_path
sa_param["data_path"] = data_path

sa_process_result = ResultParser(method, **sa_param)
logging.debug(f"{method} result is {sa_process_result.get_all_result()}")

local_time, _ , _, _= sa_process_result.get_time()

logging.info(f"time for {method}: \n \
    local time is {local_time}")

INFO:root:load simulated annealer raw result
INFO:root:valid var for this model is ['x_3_1', 'x_3_2', 'x_3_3', 'x_3_4']
INFO:root:parse simulated annealer result
INFO:root:sa only has local_time!
INFO:root:time for dwave-sa: 
     local time is 0.017264842987060547


In [4]:
sa_atom_pos_data = sa_process_result.generate_optimize_pts()
# save unfold file for visualization and parameters for experiment: 1. volume value 2. relative improvement
sa_process_result.save_mol_file(f"{timestamp}")

INFO:root:finish update optimize points for ['x_3_4']
INFO:root:finish save ./molecule-data/Aspirin_dwave-sa_20211207-11.mol2 and ./molecule-data/Aspirin_dwave-sa_20211207-11.json


x_3_4


0

In [5]:
sa_process_result.parameters

{'volume': {'initial': 726.8863198728677,
  'optimize': 709.2589220567003,
  'gain': 0.9757494434353222}}

In [6]:
method = "dwave-qa"
qa_param = {}
qa_param["bucket"] = s3_bucket
qa_param["prefix"] = prefix
qa_param["task_id"] = qa_task_id
qa_param["raw_path"] = raw_path
qa_param["data_path"] = data_path

qa_process_result = ResultParser(method, **qa_param)
logging.debug(f"{method} result is {qa_process_result.get_all_result()}")

local_time, task_time, total_time, access_time = qa_process_result.get_time()

logging.info(f"time for {method}: \n \
    local time is {local_time},\n \
    task time is {task_time}, \n \
    qpu total time is {total_time}, \n \
    qpu access time is {access_time}")

INFO:root:load quantum annealer raw result
INFO:root:valid var for this model is ['x_3_1', 'x_3_2', 'x_3_3', 'x_3_4']
INFO:root:parse quantum annealer result
INFO:root:time for dwave-qa: 
     local time is 6.910878896713257,
     task time is 2.355, 
     qpu total time is 0.023138000000000002, 
     qpu access time is 0.008577


#### parse results

In [7]:
qa_atom_pos_data = qa_process_result.generate_optimize_pts()
# save unfold file for visualization and parameters for experiment: 1. volume value 2. relative improvement
qa_process_result.save_mol_file(f"{timestamp}")

INFO:root:finish update optimize points for ['x_3_2']
INFO:root:finish save ./molecule-data/Aspirin_dwave-qa_20211207-11.mol2 and ./molecule-data/Aspirin_dwave-qa_20211207-11.json


x_3_2


0

In [8]:
qa_process_result.parameters

{'volume': {'initial': 726.8863198728677,
  'optimize': 736.1169399760236,
  'gain': 1.0126988496698772}}

### draft place

In [82]:
type(device.properties.service.executionWindows[0].windowStartHour)

datetime.time

In [80]:
import time
gmt = time.gmtime(time.time())

diff = device.properties.service.executionWindows[0].windowStartHour.hour*60*60-gmt.tm_hour*60*60-gmt.tm_min*60-gmt.tm_sec

hour = diff//3600

minute = (diff - hour*60*60)//60

sec = diff - hour*60*60 - minute*60

print("next available time {}:{}:{}".format(hour, minute, sec))

next available time 4:34:11


In [75]:
device.properties.service.executionWindows[0].windowStartHour

datetime.time(13, 0)

In [287]:
import datetime
device = AwsDevice("arn:aws:braket:::device/qpu/ionq/ionQdevice")
exec_day = device.properties.service.executionWindows[0].executionDay.value
window_start_time = device.properties.service.executionWindows[0].windowStartHour
timedelta_hour = datetime.timedelta(hours=window_start_time.hour, minutes=window_start_time.minute)
current_time = datetime.datetime.utcnow()

diff = 0
today_day = current_time.weekday()

if exec_day == 'Weekdays':
    if today_day > 4:
        diff = 7 - today_day
elif exec_day == 'Weekend':
    if today_day < 5:
        diff = 5 - today_day

timedelta = datetime.timedelta(diff)

start_time = datetime.datetime(current_time.year, current_time.month, current_time.day) + timedelta

start_time = start_time + timedelta_hour

In [288]:
current_time

datetime.datetime(2021, 11, 22, 1, 51, 42, 975369)

In [289]:
current_time.timestamp()*1000

1637545902975.369

In [259]:
delta = start_time - current_time

day = delta.days

hour = delta.seconds//3600

minute = (delta.seconds - hour*60*60)//60

sec = delta.seconds - hour*60*60 - minute*60

print("next available time {} days {}:{}:{}".format(day, hour, minute, sec))

next available time 0 days 4:55:2


In [304]:
window = device.properties.service.executionWindows[0]
print(window)
print(window.executionDay.value)
print(window.windowStartHour.minute)
print(window.windowEndHour)

executionDay=<ExecutionDay.WEEKDAYS: 'Weekdays'> windowStartHour=datetime.time(13, 0) windowEndHour=datetime.time(2, 0)
Weekdays
0
02:00:00


In [13]:
class AwsDeviceTest:
    def __init__(self, device):
        self.__properties__ = device.properties
        self._status = device.status
    @staticmethod
    def _splitWindow(window):
        start_hour = window.windowStartHour.hour
        start_min = window.windowStartHour.minute
        end_hour = window.windowEndHour.hour
        end_min = window.windowEndHour.minute
        
        return start_hour, start_min, end_hour, end_min
    
    def _createTimeRange(date, window, available_days):
        TWENTY_FOUR_HOURS_IN_MILLISECONDS = 86400000
        # hour/min string -> numbers
        start_hour, start_min, end_hour, end_min = AwsDeviceTest._splitWindow(window)
        
        time_ranges = []
        
        for day in available_days:
            # Set the start and end times to be on the same day as the current time, then shift them to there correct day using milliseconds offset
            # Shifting after converting to milliseconds is much easier than dealing with calendar dates 
            start_utc = datetime.datetime(date.year, date.month, date.day, start_hour, start_min, 0).timestamp()*1000
            end_utc = datetime.datetime(date.year, date.month, date.day, end_hour, end_min, 0).timestamp()*1000
            
            start_utc = start_utc + TWENTY_FOUR_HOURS_IN_MILLISECONDS * (day - date.weekday())
            end_utc = end_utc + TWENTY_FOUR_HOURS_IN_MILLISECONDS * (day - date.weekday())
            
            if start_utc > end_utc:
                # If the time wraps to the next day and the day is Sunday(6) we need to wrap to Monday(0) so zero out the current weekday 
                if (day == 6):
                    wrap_start_utc = wrap_start_utc + TWENTY_FOUR_HOURS_IN_MILLISECONDS * (day - 7 - date.weekday())
                    wrap_end_utc = wrap_end_utc + TWENTY_FOUR_HOURS_IN_MILLISECONDS * (day - 6 - date.weekday())
                    
                    time_ranges.insert(0, [wrap_start_utc, wrap_end_utc])
                end_utc = end_utc + TWENTY_FOUR_HOURS_IN_MILLISECONDS
            
            time_ranges.append([start_utc, end_utc])
        
        return time_ranges
    
    def _getTimeDifference(date, window, available_days):
        # We need to get the available windows in the current week and the week after incase the next available day is a week away.
        # For example, if the current time 6 pm on Friday and the range is every Friday from 1pm-5pm, then the next range will be a next Friday at 1pm
        # TODO

        # Get current time in milliseconds
        current_time = date.timestamp() * 1000
        # Create an array of possible available ranges
        time_ranges = AwsDeviceTest._createTimeRange(date, window, available_days)
        
        for time_range in time_ranges:
            # If the current time is in the range then return 0
            if current_time > time_range[0] and current_time < time_range[1]:
                return 0
            # The ranges will be in chronological order the current time will always be either in a range or less than one
            elif current_time < time_range[0]:
                return time_range[0] - current_time

    @staticmethod
    def next_available_time(self) -> int:
        """
        Get the next available time in seconds. If the device is offline or retired, 
        The return value is -1

        Examples:
            >>> next_time_in_seconds = AwsDevice.next_available_time()

        Returns:
            int : next available time in seconds
        """
        # refresh to get the recent meta data
        # self._populate_properties(self._aws_session)
        DaysEnum = {
            'Sunday': 6,
            'Monday': 0,
            'Tuesday': 1,
            'Wednesday': 2,
            'Thursday': 3,
            'Friday': 4,
            'Saturday': 5
            }
            
        if self._status in ['OFFLINE', 'RETIRED']:
            # the device is not online
            return -1
            
        windows = self._properties.service.executionWindows
        
        # minimum time is 20 ms
        min_time = None
        calc_time = 0
        
        date = datetime.datetime.utcnow()

        for window in windows:
            exec_day = window.executionDay.value
            # "Everyday" | "Weekdays" | "Weekend" | "Monday" | "Tuesday" | "Wednesday" | "Thursday" | "Friday" | "Saturday" | "Sunday"
            if exec_day == 'Everyday':
                calc_time = AwsDevice._getTimeDifference(date, window, range(0,7))
            elif exec_day == 'Weekdays':
                calc_time = AwsDevice._getTimeDifference(date, window, range(0,6))
            elif exec_day == 'Weekend':
                calc_time = AwsDevice._getTimeDifference(date, window, [5,6])
            else:
                calc_time = AwsDevice._getTimeDifference(date, window, [DaysEnum[exec_day]])
        
        if (min_time == None or calc_time < min_time):
            min_time = calc_time
        if min_time == None:
            return 0
        else:
            return min_time/1000.

In [14]:
from braket.aws import AwsDevice
import datetime
device = AwsDevice("arn:aws:braket:::device/qpu/d-wave/Advantage_system4")
# device = AwsDevice("arn:aws:braket:::device/qpu/d-wave/Advantage_system1")
# device = AwsDevice("arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6")
# device = AwsDevice("arn:aws:braket:::device/qpu/ionq/ionQdevice")
# device = AwsDevice("arn:aws:braket:::device/qpu/rigetti/Aspen-8")
# device = AwsDevice("arn:aws:braket:::device/qpu/rigetti/Aspen-9")
# device = AwsDevice("arn:aws:braket:::device/qpu/rigetti/Aspen-10")
# device = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/sv1")
# device = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/tn1")
# device = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/dm1")
awsdevice = AwsDeviceTest(device)

min_time = awsdevice.next_available_time()

hour = min_time//3600

minute = (min_time - hour*60*60)//60

sec = min_time - hour*60*60 - minute*60

print("next available time {}:{}:{}".format(hour, minute, sec))


INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


TypeError: next_available_time() missing 1 required positional argument: 'self'

In [17]:
awsdevice = AwsDeviceTest(device)
awsdevice.next_available_time()

TypeError: next_available_time() missing 1 required positional argument: 'self'

In [382]:
device = AwsDevice("arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6")
min_time = getNextAvailable(device)

hour = min_time//3600

minute = (min_time - hour*60*60)//60

sec = min_time - hour*60*60 - minute*60

print("next available time {}:{}:{}".format(hour, minute, sec))

calc time 0
next available time 0.0:0.0:0.0


In [366]:
device.properties.service.executionWindows[0]

DeviceExecutionWindow(executionDay=<ExecutionDay.WEEKDAYS: 'Weekdays'>, windowStartHour=datetime.time(13, 0), windowEndHour=datetime.time(2, 0))

In [346]:
date = datetime.datetime.utcnow()

In [352]:
start_hour = window.windowStartHour.hour
start_min = window.windowStartHour.minute
end_hour = window.windowEndHour.hour
end_min = window.windowEndHour.minute

start_utc = datetime.datetime(date.year, date.month, date.day, start_hour, start_min, 0)
end_utc = datetime.datetime(date.year, date.month, date.day, end_hour, end_min, 0)
start_utc.weekday()

0

In [257]:
today_day

4

In [102]:
import datetime

a = datetime.datetime.utcnow()
b = datetime.datetime(1970, 1, 1)
a -b

datetime.timedelta(days=18949, seconds=31654, microseconds=708438)

In [110]:
(today.weekday() + 6) % 7

4

In [109]:
today = datetime.datetime(2021, 11, 13)


offset = max(0, (today.weekday() + 6) % 7 - 3)


timedelta = datetime.timedelta(offset)


most_recent = today + timedelta


print(most_recent)

2021-11-14 00:00:00
